<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_function_call.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto Generated Agent Chat: Task Solving with Provided Tools as Functions

AutoGen offers conversable agents powered by LLM, tool, or human, which can be used to perform tasks collectively via automated chat. This framework allows tool use and human participation through multi-agent conversation. Please find documentation about this feature [here](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat).

In this notebook, we demonstrate how to use `NexusFunctionCallingAssistant` and `UserProxyAgent` to make function calls with the new feature of OpenAI models (in model version 0613). A specified prompt and function configs must be passed to `NexusFunctionCallingAssistant` to initialize the agent. The corresponding functions must be passed to `UserProxyAgent`, which will execute any function calls made by `NexusFunctionCallingAssistant`. Besides this requirement of matching descriptions with functions, we recommend checking the system message in the `NexusFunctionCallingAssistant` to ensure the instructions align with the function call descriptions.

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install `pyautogen`:
```bash
pip install pyautogen
```

In [1]:
# %pip install "pyautogen>=0.2.3"

## Import Required Libraries

In [ ]:
import os
from typing import Literal
from typing_extensions import Annotated
import autogen
import autogen.agentchat.contrib.nexusravenv2_local_function_calling as Nexus

## Install Ollama

In [ ]:
if os.name == "posix":  # Linux or macOS
    !curl -fsSL https://ollama.com/install.sh | sh
elif os.name == "nt":  # Windows
    !winget install Ollama.Ollama

## Run Required Library
NexusRaven is a standard [Ollama Library](https://ollama.com/library/nexusraven)
Ollama binds 127.0.0.1 port 11434 by default.
Change the bind address with the OLLAMA_HOST environment variable. 

In [ ]:
%ollama run nexusraven

## Set your API Endpoint

Check the ollama console for the network address.

In [ ]:
network_address = "127.0.0.1:11434"  # this may change please check the output above

llm_config = {
    "config_list": [{"model": "litellmnotneeded", "api_key": "NotRequired", "base_url": f"http://{network_address}"}],
    "cache_seed": None,
}  ## CRITICAL - ENSURE THERE'S NO CACHING FOR TESTING

In [ ]:
chatbot = Nexus.NexusFunctionCallingAssistant(
    name="chatbot",
    # Iteration 4, two examples to help guide LLM on response
    system_message="""For currency exchange tasks, 
        only use the functions you have been provided with. 
        Output 'BAZINGA!' when an answer has been provided. 
        Do not include the function name or result in the JSON.
        Example of the return JSON is: 
        {
            "parameter_1_name": 100.00,
            "parameter_2_name": "ABC",
            "parameter_3_name": "DEF",
        }. 
        Another example of the return JSON is: 
        {
            "parameter_1_name": "GHI",
            "parameter_2_name": "ABC",
            "parameter_3_name": "DEF",
            "parameter_4_name": 123.00,
        }. """,  # MS - this was needed to ensure the function name was returned
    llm_config=llm_config,
)

# create a UserProxyAgent ins

In [2]:
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    # MS updated to search for BAZINGA! to terminate
    is_termination_msg=lambda x: x.get("content", "")
    and "BAZINGA!" in x.get("content", ""),  # and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=4,
)

## Making Function Calls

In this example, we demonstrate function call execution with `NexusFunctionCallingAssistant` and `UserProxyAgent`. We allow the LLM assistant to perform tasks with code, and the `UserProxyAgent` would extract code blocks from the LLM response and execute them. With the new "function_call" feature, we define functions and specify the description of the function in the OpenAI config for the `NexusFunctionCallingAssistant`. Then we register the functions in `UserProxyAgent`.


In [3]:
# define functions according to the function description

# one way of registering functions is to use the register_for_llm and register_for_execution decorators
CurrencySymbol = Literal["USD", "EUR"]


def exchange_rate(base_currency: CurrencySymbol, quote_currency: CurrencySymbol) -> float:
    if base_currency == quote_currency:
        return 1.0
    elif base_currency == "USD" and quote_currency == "EUR":
        return 1 / 1.1
    elif base_currency == "EUR" and quote_currency == "USD":
        return 1.1
    else:
        raise ValueError(f"Unknown currencies {base_currency}, {quote_currency}")

In [ ]:
@user_proxy.register_for_execution()
@chatbot.register_for_llm(description="Currency exchange calculator.")
def currency_calculator(
    base_amount: Annotated[float, "Amount of currency in base_currency"],
    base_currency: Annotated[CurrencySymbol, "Base currency"] = "USD",
    quote_currency: Annotated[CurrencySymbol, "Quote currency"] = "EUR",
) -> str:
    quote_amount = exchange_rate(base_currency, quote_currency) * base_amount
    return f"{format(quote_amount, '.2f')} {quote_currency}"

Finally, we initialize the chat that would use the functions defined above:

In [ ]:
res = user_proxy.initiate_chat(
    chatbot,
    message="How much is 123.45 EUR in USD?",
    summary_method="reflection_with_llm",
)